# ECON 320 — Econometrics Lab
## Data Loading & Formatting Workshop (Group Work)

> Work with your **research-proposal groups**. Today you will: (1) load your data, (1B) reshape panel data if needed, (1C) merge files if needed, (2) write your model equation in LaTeX, (3) estimate OLS, and (4) present results with a publication-style table.

### 👥 Group Members (3–5 people)
List names with lab in parentheses.

**Example:**
- Name 1 (**Lab 3**)
- Name 2 (**Lab 3**)
- Name 3 (**Lab 3**)
- Name 4 (**Lab 3**) *(optional)*
- Name 5 (**Lab 3**) *(optional)*

**Your group:**
- Howie Brown
- Eshan Momin
- Ashley Mazarin
- Caleigh Shapiro
- 

In [7]:
# 🧰 Setup — common packages (you can add more as needed)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

try:
    from stargazer.stargazer import Stargazer
    HAVE_STARGAZER = True
except Exception:
    HAVE_STARGAZER = False
    print('Note: stargazer not available; will fall back to summary_col. Install with `pip install stargazer` if desired.')

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 140)
%matplotlib inline

In [6]:
pip install stargazer

  Obtaining dependency information for stargazer from https://files.pythonhosted.org/packages/af/49/0f0bc34f94e6ba65f8e5cc06208d11759153f02fb61f4a9f9db98e86bb68/stargazer-0.0.7-py3-none-any.whl.metadata
  Using cached stargazer-0.0.7-py3-none-any.whl.metadata (6.3 kB)
Using cached stargazer-0.0.7-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


## ✍️ Exercise 1 — Load your dataset into a pandas DataFrame
- Load the dataset you intend to use for the final project into a DataFrame named **`df`**.
- Columns should include your **outcome (Y)** and **regressors (X's)**.
- Reuse import code from the previous handout if applicable; otherwise write new code (LLM help is fine).

In [12]:
# ⬇️ Replace with your actual path/loader and assign to df
data_path = pd.read_csv("data_cleaned.csv")  # e.g., 'data/myfile.csv'

# Examples — uncomment and edit ONE:
# df = pd.read_csv(data_path)
# df = pd.read_excel(data_path, sheet_name=0)
# df = pd.read_json(data_path)
# df = pd.read_parquet(data_path)

try:
    display(df.head()); print('\nShape:', df.shape)
except NameError:
    print('Define your DataFrame as df, then re-run.')

Define your DataFrame as df, then re-run.


In [19]:
df = pd.read_csv("data_cleaned.csv")

In [20]:
df

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STRATA,GQ,HHINCOME,VALUEH,PERNUM,PERWT,AGE,EDUC,EDUCD
0,2023,202301,3710,2023000000061,69.0,2023000037101,200001,1,11000,100000,1,69.0,54,7,71
1,2023,202301,3710,2023000000061,69.0,2023000037101,200001,1,11000,100000,2,54.0,35,4,40
2,2023,202301,3710,2023000000061,69.0,2023000037101,200001,1,11000,100000,3,67.0,14,3,30
3,2023,202301,3712,2023000000328,126.0,2023000037121,230001,1,111500,80000,1,126.0,40,7,71
4,2023,202301,3712,2023000000328,126.0,2023000037121,230001,1,111500,80000,2,146.0,41,6,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439250,2023,202301,1519007,2023001456541,125.0,2023015190071,50056,1,76700,320000,3,69.0,42,10,101
2439251,2023,202301,1519007,2023001456541,125.0,2023015190071,50056,1,76700,320000,4,68.0,38,6,64
2439252,2023,202301,1519008,2023001457972,77.0,2023015190081,30056,1,145000,388000,1,78.0,53,11,114
2439253,2023,202301,1519008,2023001457972,77.0,2023015190081,30056,1,145000,388000,2,62.0,66,10,101


## 🔁 Exercise 1B (only if you have panel data: entities and years)
**Skip if you do not have a time component.** If your table has **rows = entities** (e.g., countries) and **columns = years** (e.g., 2000, 2001, …), reshape to a standard panel format with one row per **entity–year**.

### Sample reshape — `melt`
Assume `df_wide` has `country` and year columns (e.g., `2000, 2001, ...`) storing a variable (e.g., `wage`).

In [14]:
# --- wide -> long with melt ---
df_wide = None  # set this to your wide-format DataFrame
if df_wide is not None:
    year_cols = [c for c in df_wide.columns if str(c).isdigit()]
    df_long = df_wide.melt(id_vars=['country'], value_vars=year_cols,
                           var_name='year', value_name='wage')
    df_long['year'] = df_long['year'].astype(int)
    display(df_long.head())
else:
    print('Set df_wide to your wide table to run this example.')

Set df_wide to your wide table to run this example.


### Sample reshape — split names with suffix years
If columns are like `wage_2000, wage_2001, price_2000, ...`, split names to extract the year and variable.

In [ ]:
# --- variables with year suffix -> long ---
df_wide2 = None  # set to your DataFrame
if df_wide2 is not None:
    long_frames = []
    for col in df_wide2.columns:
        if '_' in col and col.split('_')[-1].isdigit():
            base, yr = col.rsplit('_', 1)
            tmp = df_wide2[['country', col]].copy()
            tmp.rename(columns={col: base}, inplace=True)
            tmp['year'] = int(yr)
            long_frames.append(tmp)
    df_long2 = pd.concat(long_frames, ignore_index=True).drop_duplicates()
    display(df_long2.sort_values(['country','year']).head())
else:
    print('Set df_wide2 to your wide table to run this example.')

After reshaping, you should have tidy panel data:

`country | year | outcome | regressors ...`

Ensure `year` is numeric and sorted; optionally set `['country','year']` as an index.

In [ ]:
# (Optional) set a multi-index and sort
# df = df_long  # or df_long2
# df = df.sort_values(['country','year']).set_index(['country','year'])
# df.head()

## ➕ Exercise 1C (only if you need to merge several files)
**Skip if you do not need to merge.** Merge by common keys (e.g., `country`, `year`).

In [ ]:
# --- Merge example ---
df_a, df_b = None, None  # set these to your DataFrames
if df_a is not None and df_b is not None:
    df_merged = pd.merge(df_a, df_b, on=['country','year'], how='inner')
    display(df_merged.head())
else:
    print('Set df_a and df_b to your DataFrames to run this example.')

### Our dataset does not have year based variables and as such we do not need to do 1B. Additionally, we do not have multiple files to merge, we have already gotten the entire dataset.

## 🧪 Exercise 2 — Write your main regression equation (LaTeX)
Write the equation that summarizes your research question.

**Cross-section:**
\[
\texttt{wage}_i = \beta_0 + \beta_1\,\texttt{education}_i + \beta_2\,\texttt{experience}_i + \varepsilon_i
\]
$i$ indexes individuals (or firms, countries, etc.).

**Panel:**
\[
\texttt{wage}_{it} = \beta_0 + \beta_1\,\texttt{education}_{it} + \beta_2\,\texttt{experience}_{it} + \varepsilon_{it}
\]
$i$ indexes the entity; $t$ is the time period. Replace names with your actual variables (`outcome_{it}`, `X1_{it}`, ...).

### Main Regression Equation

Our hypothesis states that households with higher income and higher education levels tend to own homes with greater values, controlling for age.

We test this relationship using the following regression model:

\[
\texttt{VALUEH}_{it} = \beta_0 + \beta_1 \texttt{HHINCOME}_{it} + \beta_2 \texttt{EDUC}_{it} + \beta_3 \texttt{AGE}_{it} + \varepsilon_{it}
\]

where:
- \( i \) indexes the household,
- \( t \) indexes the survey year,
- \(\texttt{VALUEH}\) is the value of the owned home (dependent variable),
- \(\texttt{HHINCOME}\) is household income,
- \(\texttt{EDUC}\) is the education level of the household head,
- \(\texttt{AGE}\) is the age of the household head (control variable),
- and \(\varepsilon_{it}\) is the error term capturing unobserved factors such as neighborhood quality or credit access.

This model investigates a **correlation** rather than a **causal** relationship, as unobserved factors may influence both income and home value.


## 📈 Exercise 3 — Estimate the OLS model
- Choose your outcome `Y` and regressors `X1, X2, …`.
- **Panel data**: for this exercise, keep a **single year** with the most observations to run a simple OLS.
- Print the results table.

In [ ]:
# Set variables
Y = 'REPLACE_WITH_Y'
X = ['X1','X2']

# If panel, keep the most frequent year
# year_col = 'year'
# top_year = df[year_col].value_counts().idxmax()
# df_ols = df[df[year_col] == top_year].copy()
# print(f'Using year {top_year} for this OLS example.')

df_ols = df.copy()
formula = Y + ' ~ ' + ' + '.join(X)
model = smf.ols(formula=formula, data=df_ols).fit()
display(model.summary())

In [21]:
# --- Exercise 3: Estimate the OLS model ---

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

# 1) Set outcome and regressors from your hypothesis
Y = 'VALUEH'
X = ['HHINCOME', 'EDUC', 'AGE']

# 2) Keep a single year (the one with the most observations)
df_ols = df.copy()
year_col = 'YEAR'
top_year = df_ols[year_col].mode()[0]  # most frequent year
df_ols = df_ols[df_ols[year_col] == top_year].copy()
print(f'Using year {top_year} for this OLS example.')

# 3) Keep only needed columns and drop rows with missing values
keep_cols = [Y] + X
df_ols = df_ols[keep_cols].dropna()
print(f'Rows used: {len(df_ols):,}')

# 4) Plain-levels model (matches your written equation)
formula = f"{Y} ~ " + " + ".join(X)
model = smf.ols(formula=formula, data=df_ols).fit(cov_type="HC3")  # robust SEs
display(model.summary())

# ---------- OPTIONAL: log specification (uncomment to try) ----------
# # Only keep positive values for logs
# df_log = df_ols[(df_ols[Y] > 0) & (df_ols['HHINCOME'] > 0)].copy()
# df_log['ln_VALUEH']  = np.log(df_log['VALUEH'])
# df_log['ln_HHINCOME'] = np.log(df_log['HHINCOME'])
# log_formula = "ln_VALUEH ~ ln_HHINCOME + EDUC + AGE"
# log_model = smf.ols(formula=log_formula, data=df_log).fit(cov_type="HC3")
# print("\n--- Log-linear model (elasticity of home value wrt income) ---")
# display(log_model.summary())


Using year 2023 for this OLS example.
Rows used: 2,439,255


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 VALUEH   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                 3.352e+04
Date:                Fri, 17 Oct 2025   Prob (F-statistic):               0.00
Time:                        11:52:10   Log-Likelihood:            -3.5827e+07
No. Observations:             2439255   AIC:                         7.165e+07
Df Residuals:                 2439251   BIC:                         7.165e+07
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.673e+05   1193.886    140.126      0.000    1.65e+05     1.7e+05
HHINCOME       1.7300      0.006    273.307      0.000       1.718       1.742
EDUC        7975.3246    144.537     55.178      0.000    7692.037    8258.612
AGE          270.1752     18.910     14.287      0.000     233.111     307.239
==============================================================================
Omnibus:                  2889458.732   Durbin-Watson:                   0.689
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        410547979.588
Skew:                           6.303   Prob(JB):                         0.00
Kurtosis:                      65.294   Cond. No.                     5.32e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## 🗂️ Exercise 4 — Present results with a publication-style table (Stargazer)
If available, use `stargazer` to render a LaTeX/HTML table; otherwise use the tidy fallback.


In [ ]:
if 'model' in globals():
    if HAVE_STARGAZER:
        sg = Stargazer([model])
        sg.title('OLS Results')
        print(sg.render_latex())
        # For HTML: print(sg.render_html())
    else:
        from statsmodels.iolib.summary2 import summary_col
        print('Stargazer unavailable — showing compact table via summary_col:')
        print(summary_col([model], stars=True, model_names=['Baseline']).as_text())
else:
    print('Run Exercise 3 first to create `model`.')

In [23]:
# --- Exercise 4 (display like the screenshot): HTML preview + LaTeX text ---

from IPython.display import HTML, display

if 'model' not in globals():
    raise RuntimeError("Run Exercise 3 first to create `model`.")

# Try to use stargazer; otherwise fallback
try:
    from stargazer.stargazer import Stargazer
    HAVE_STARGAZER = True
except ImportError:
    HAVE_STARGAZER = False

if HAVE_STARGAZER:
    sg = Stargazer([model])
    sg.title('OLS Results — Home Value vs Income & Education')
    sg.custom_columns(['Baseline (HC3 SEs)'], [1])

    # Optional niceties
    sg.dependent_variable_name('Home Value (VALUEH)')
    sg.covariate_order(['HHINCOME', 'EDUC', 'AGE'])
    sg.rename_covariates({
        'HHINCOME':'Household Income',
        'EDUC':'Education (years/level)',
        'AGE':'Age'
    })

    # In-notebook HTML preview (like your screenshot)
    display(HTML(sg.render_html()))

    # Also print LaTeX so you can paste into Overleaf if needed
    print("\nLaTeX version (copy into Overleaf if needed):\n")
    print(sg.render_latex())

else:
    # Fallback: compact, publication-style text table
    from statsmodels.iolib.summary2 import summary_col
    print("Stargazer not available — showing compact table via summary_col:\n")
    tbl = summary_col(
        [model],
        stars=True,
        float_format='%0.3f',
        model_names=['Baseline (HC3)'],
        info_dict={'N':lambda x: f"{int(x.nobs)}",
                   'R2':lambda x: f"{x.rsquared:.3f}",
                   'Adj. R2':lambda x: f"{x.rsquared_adj:.3f}"}
    )
    display(HTML(f"<pre>{tbl.as_text()}</pre>"))



LaTeX version (copy into Overleaf if needed):

\begin{table}[!htbp] \centering
  \caption{OLS Results — Home Value vs Income & Education}
\begin{tabular}{@{\extracolsep{5pt}}lc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{1}{c}{\textit{Dependent variable: Home Value (VALUEH)}} \
\cr \cline{2-2}
\\[-1.8ex] & \multicolumn{1}{c}{Baseline (HC3 SEs)}  \\
\\[-1.8ex] & (1) \\
\hline \\[-1.8ex]
 Household Income & 1.730$^{***}$ \\
& (0.006) \\
 Education (years/level) & 7975.325$^{***}$ \\
& (144.537) \\
 Age & 270.175$^{***}$ \\
& (18.910) \\
\hline \\[-1.8ex]
 Observations & 2439255 \\
 $R^2$ & 0.150 \\
 Adjusted $R^2$ & 0.150 \\
 Residual Std. Error & 578829.100 (df=2439251) \\
 F Statistic & 33520.423$^{***}$ (df=3; 2439251) \\
\hline
\hline \\[-1.8ex]
\textit{Note:} & \multicolumn{1}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\
\end{tabular}
\end{table}


### ✅ Suggestions
- Keep variable names simple (lowercase, underscores).
- Create a small **data dictionary** in a markdown cell.
- If merges fail, check key types and spellings; try `how='inner'` first.
- Quick missingness scan: `df.isna().mean().sort_values(ascending=False).head(10)`.
- Save intermediate datasets (CSV/Parquet) so you can resume quickly next time.

## 📄 Example — Stargazer table (simulated data)
This example simulates a simple dataset and renders a results table with `stargazer` so you can see the expected look.
If `stargazer` isn't installed in your environment, the notebook will show a tidy fallback table instead.


In [2]:
import numpy as np
from IPython.display import HTML, display

# Simulate data
np.random.seed(0)
n = 500
X1 = np.random.normal(size=n)
X2 = np.random.normal(size=n)
eps = np.random.normal(scale=1.0, size=n)
Y = 1.0 + 0.5*X1 - 0.3*X2 + eps

df_sim = pd.DataFrame({'Y': Y, 'X1': X1, 'X2': X2})
model_sim = smf.ols('Y ~ X1 + X2', data=df_sim).fit()

# Render with Stargazer if available; otherwise show a compact fallback
if 'HAVE_STARGAZER' in globals() and HAVE_STARGAZER:
    sg = Stargazer([model_sim])
    sg.title('Simulated OLS Example')
    # HTML preview in the notebook
    display(HTML(sg.render_html()))
    # Also print LaTeX source so you can copy it into Overleaf if desired
    print('\nLaTeX version (copy into Overleaf if needed):\n')
    print(sg.render_latex())
else:
    from statsmodels.iolib.summary2 import summary_col
    print('Stargazer not available — showing compact table via summary_col:')
    display(summary_col([model_sim], stars=True, model_names=['Simulated']))



LaTeX version (copy into Overleaf if needed):

\begin{table}[!htbp] \centering
  \caption{Simulated OLS Example}
\begin{tabular}{@{\extracolsep{5pt}}lc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{1}{c}{\textit{Dependent variable: Y}} \
\cr \cline{2-2}
\\[-1.8ex] & (1) \\
\hline \\[-1.8ex]
 Intercept & 1.027$^{***}$ \\
& (0.043) \\
 X1 & 0.455$^{***}$ \\
& (0.043) \\
 X2 & -0.311$^{***}$ \\
& (0.044) \\
\hline \\[-1.8ex]
 Observations & 500 \\
 $R^2$ & 0.250 \\
 Adjusted $R^2$ & 0.247 \\
 Residual Std. Error & 0.967 (df=497) \\
 F Statistic & 82.683$^{***}$ (df=2; 497) \\
\hline
\hline \\[-1.8ex]
\textit{Note:} & \multicolumn{1}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\
\end{tabular}
\end{table}
